# Input image and drawing two lines

In [ ]:
import cv2
import math
import numpy as np
from matplotlib import pyplot as plt
from math import sqrt
from math import acos, degrees


kernel = np.ones((5,5),np.uint8)
img1 = cv2.imread('n.jpg')
img = cv2.imread('n.jpg',0)
gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

ret, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

# Create mask
height,width = img.shape
#height=height-10
#width=width-10
mask = np.zeros((height,width), np.uint8)

edges = cv2.Canny(thresh, 100, 200)

#cv2.imshow('detected ',gray)
cimg=cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)
#circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1.2, 1000, param1 = 50, param2 = 30, minRadius = 20, maxRadius = 0)
for i in circles[0,:]:
    i[2]=i[2]+4
    # Draw on mask
    cv2.circle(mask,(i[0],i[1]),i[2],(255,255,255),thickness=-1)

# Copy that image using that mask
masked_data = cv2.bitwise_and(img1, img1, mask=mask)

# Apply Threshold
_,thresh = cv2.threshold(mask,1,255,cv2.THRESH_BINARY)
# Find Contour
contours = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
x,y,w,h = cv2.boundingRect(contours[0])

# Crop masked_data
crop = masked_data[y+30:y+h-30,x+30:x+w-30]
i=crop
height, width, channels = i.shape
print (width, height, channels)
#########################################################################

ret, mask = cv2.threshold(i, 10, 255, cv2.THRESH_BINARY)
edges = cv2.Canny(i,100,200)
kernel = np.ones((11,11),np.uint8)
kernel2 = np.ones((13,13),np.uint8)
edges = cv2.dilate(edges,kernel,iterations = 1)
edges = cv2.erode(edges,kernel2,iterations = 1)
minLineLength = 1000
maxLineGap = 10
lines = cv2.HoughLinesP(edges,1,np.pi/180,15,minLineLength,maxLineGap)
h=[]
xmax1=0
xmax2=0
ymax1=0
ymax2=0
xs1=0
xs2=0
ys1=0
ys2=0

for line in lines:
    x1, y1, x2, y2 = line[0]
    #cv2.line(i, (x1, y1), (x2, y2), (0, 255, 0), 1)
    dx=x2-x1
    if(dx<0):
        dx=dx*-1
    dy=y2-y1
    if(dy<0):
        dy=dy*-1
        
    hypo=sqrt(dx**2 + dy**2)
    
            
    #print("dx=",dx,"  dy=",dy)
    h.append(hypo)

#print(h)
print(len(h))
a=len(h)
h.sort(reverse=True)
#print(h)
m=0
k=0

for f in range(a):
    for line in lines:
        x1, y1, x2, y2 = line[0]
        #cv2.line(i, (x1, y1), (x2, y2), (0, 255, 0), 3)
        dx=x2-x1
        if(dx<0):
            dx=dx*-1
        dy=y2-y1
        if(dy<0):
            dy=dy*-1

        hypo2=sqrt(dx**2 + dy**2)


        if(hypo2==h[0]):
            m=hypo2
            xmax1=x1
            xmax2=x2
            ymax1=y1
            ymax2=y2
            cv2.line(crop, (xmax1, ymax1), (xmax2, ymax2), (255, 0, 0), 3)
            #print("xmax1=",xmax1," ymax1=",ymax1," xmax2=",xmax2," ymax2=",ymax2)

        if(m==h[0]): 
            if(hypo2==h[f]):
                if((sqrt((xmax2-x2)**2 + (ymax2-y2)**2))>20):
                    if((sqrt((xmax1-x1)**2 + (ymax1-y1)**2))>20):
                        xs1=x1
                        xs2=x2
                        ys1=y1
                        ys2=y2
                        cv2.line(crop, (xs1, ys1), (xs2, ys2), (0, 255, 0), 3)
                        print("xs1=",xs1," ys1=",ys1," xs2=",xs2," ys2=",ys2)
                        k=1
                        break
    if(k==1):                
        break           

print("xmax1=",xmax1," ymax1=",ymax1," xmax2=",xmax2," ymax2=",ymax2)



# Algorithm of clock calculation

## 1-Calculate center point

In [ ]:
xcenter=width/2
ycenter=height/2
print('xcenter=' ,xcenter,"ycenter=",ycenter)

## 2-Determine the coordinates of the end point (farther from the center)


In [ ]:
hassona1=abs(xcenter-xs1)
hassona2=abs(xcenter-xs2)
abdo1=abs(xcenter-xmax1)
abdo2=abs(xcenter-xmax2)
if(hassona1>hassona2):
    xhours=xs1
    yhours=ys1
else:
    xhours=xs2
    yhours=ys2
    
if(abdo1>abdo2):
    xminutes=xmax1
    yminutes=ymax1
else:
    xminutes=xmax2
    yminutes=ymax2  
    
print('xhours=' ,xhours,"yhours=",yhours,'xminutes=' ,xminutes,"yminutes=",yminutes)    

## 3- Calculate theta of each line using the law of cosines 

In [ ]:
#calculating theta for hours hand
l1=sqrt(((xcenter-xhours)**2)+((ycenter-yhours)**2))
l2=ycenter
l3=sqrt(((xcenter-xhours)**2)+((0-yhours)**2))
cos_theta_hours=(((l1)**2)+((l2)**2)-((l3)**2))/(2*(l1)*(l2))

theta_hours_radian=acos(cos_theta_hours)
theta_hours=math.degrees(theta_hours_radian)
print('theta_hours=' ,theta_hours)

#calculating theta for minutes hand
len1=sqrt(((xcenter-xminutes)**2)+((ycenter-yminutes)**2))
len2=ycenter
len3=sqrt(((xcenter-xminutes)**2)+((0-yminutes)**2))
cos_theta_minutes=(((len1)**2)+((len2)**2)-((len3)**2))/(2*(len1)*(len2))

theta_minutes_radian=acos(cos_theta_minutes)
theta_minutes=math.degrees(theta_minutes_radian)
print('theta_minutes=' ,theta_minutes)

## 4-calculate hours


In [ ]:
#Determine left half or right half
if(xhours>xcenter):
    right=1
else:
    right=0
    
if(right==1):
    hournum=int(theta_hours/30)
if(right==0):
    hournum=12-(int(theta_hours/30))
    
if(hournum==0):
    hournum=12
print('hours=' ,hournum)

## 5- calculate minutes 

In [ ]:
if(xminutes>xcenter):
    right1=1
else:
    right1=0
    
if(right1==1):
    minutesnum=int(theta_minutes/6)
if(right1==0):
    minutesnum=60-(int(theta_minutes/6))
    if(xminutes==xcenter):
        minutesnum=30
    
print('minutesnum=' ,minutesnum)



## Testing

In [ ]:
#Code to close Window
if ((minutesnum<10))
    print("TIME =  " ,hournum,": 0",minutesnum)
else     
    print("TIME =  " ,hournum,":",minutesnum)
    
cv2.imshow('Input Image',img1)
cv2.imshow('canny edge',edges)
cv2.imshow('output cropped image',crop)
cv2.waitKey(0)
cv2.destroyAllWindows()